In [1]:
import torch

import cv2
import wandb
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from datasets import load_metric

from torch import nn
import torch.nn as nn
from torchmetrics.classification import BinaryJaccardIndex
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from glob import glob
import torch.optim as optim
import pytorch_lightning as pl
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.optim.lr_scheduler import CosineAnnealingLR

In [2]:
class segDataset(Dataset):
    def __init__(self, data_path, transforms=None):
        # cata path 설정 잘하기
        self.pos_imgs = sorted(glob(data_path + 'Positive/Image/*'))
        self.pos_labels = sorted(glob(data_path + 'Positive/Label/*'))
        self.neg_imgs = sorted(glob(data_path + 'Negative/Image/*'))
        self.neg_labels = sorted(glob(data_path + 'Negative/Label /*'))
        # positive와 negative를 합쳐서 불러오는 코드를 작성
        self.imgs = self.pos_imgs + self.neg_imgs
        self.labels = self.pos_labels + self.neg_labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, item):
        img_path = self.imgs[item]
        label_path = self.labels[item]
        img = cv2.imread(img_path)
        label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)
        label = np.expand_dims(label, axis=2)

        concat = np.concatenate([img, label], axis=2)
        concat = torch.from_numpy(concat)
        concat = concat.permute(2,0,1) # (h,w,c -> c,h,w)

        
        if self.transforms:
            imgs = self.transforms(concat)

        X = imgs[:3].to(torch.float32)
        y = imgs[3].to(torch.float32)
            
        return {'X' : X/255, 'y': y/255}



data_path = './data/Train/'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    # transforms.ToTensor(), # (h,w,c -> c,h,w) + Normalize
])

training_data = segDataset(data_path=data_path, transforms=transform)

total_samples = len(training_data)
train_size = int(0.8 * total_samples)
val_size = total_samples - train_size

# 인덱스를 무작위로 섞음
indices = list(range(total_samples))
np.random.shuffle(indices)
train_sampler = SubsetRandomSampler(indices[:train_size])
val_sampler = SubsetRandomSampler(indices[train_size:])


# DataLoader 설정
train_dataloader = DataLoader(training_data, batch_size=16, sampler=train_sampler)
val_dataloader = DataLoader(training_data, batch_size=16, sampler=val_sampler)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from modeling.sync_batchnorm.batchnorm import SynchronizedBatchNorm2d
from modeling.aspp import build_aspp
from modeling.decoder import build_decoder
from modeling.backbone import build_backbone

In [4]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, rate=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               dilation=rate, padding=rate, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.rate = rate

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, nInputChannels, block, layers, os=16, pretrained=False):
        self.inplanes = 64
        super(ResNet, self).__init__()
        if os == 16:
            strides = [1, 2, 2, 1]
            rates = [1, 1, 1, 2]
            blocks = [1, 2, 4]
        elif os == 8:
            strides = [1, 2, 1, 1]
            rates = [1, 1, 2, 2]
            blocks = [1, 2, 1]
        else:
            raise NotImplementedError

        # Modules
        self.conv1 = nn.Conv2d(nInputChannels, 64, kernel_size=7, stride=2, padding=3,
                                bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=strides[0], rate=rates[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=strides[1], rate=rates[1])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=strides[2], rate=rates[2])
        self.layer4 = self._make_MG_unit(block, 512, blocks=blocks, stride=strides[3], rate=rates[3])

        self._init_weight()

        if pretrained:
            self._load_pretrained_model()

    def _make_layer(self, block, planes, blocks, stride=1, rate=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, rate, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _make_MG_unit(self, block, planes, blocks=[1,2,4], stride=1, rate=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, rate=blocks[0]*rate, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, len(blocks)):
            layers.append(block(self.inplanes, planes, stride=1, rate=blocks[i]*rate))

        return nn.Sequential(*layers)

    def forward(self, input):
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        low_level_feat = x
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x, low_level_feat

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _load_pretrained_model(self):
        pretrain_dict = model_zoo.load_url('https://download.pytorch.org/models/resnet101-5d3b4d8f.pth')
        model_dict = {}
        state_dict = self.state_dict()
        for k, v in pretrain_dict.items():
            if k in state_dict:
                model_dict[k] = v
        state_dict.update(model_dict)
        self.load_state_dict(state_dict)

def ResNet101(nInputChannels=3, os=16, pretrained=False):
    model = ResNet(nInputChannels, Bottleneck, [3, 4, 23, 3], os, pretrained=pretrained)
    return model


class ASPP_module(nn.Module):
    def __init__(self, inplanes, planes, rate):
        super(ASPP_module, self).__init__()
        if rate == 1:
            kernel_size = 1
            padding = 0
        else:
            kernel_size = 3
            padding = rate
        self.atrous_convolution = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                                            stride=1, padding=padding, dilation=rate, bias=False)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.atrous_convolution(x)
        x = self.bn(x)

        return self.relu(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


class DeepLabv3_plus(pl.LightningModule):
    def __init__(self, nInputChannels=3, n_classes=21, os=16, pretrained=False, _print=True):
        if _print:
            print("Constructing DeepLabv3+ model...")
            print("Number of classes: {}".format(n_classes))
            print("Output stride: {}".format(os))
            print("Number of Input Channels: {}".format(nInputChannels))
        super(DeepLabv3_plus, self).__init__()
        self.jaccard = BinaryJaccardIndex()
        # Atrous Conv
        self.resnet_features = ResNet101(nInputChannels, os, pretrained=pretrained)

        # ASPP
        if os == 16:
            rates = [1, 6, 12, 18]
        elif os == 8:
            rates = [1, 12, 24, 36]
        else:
            raise NotImplementedError

        self.aspp1 = ASPP_module(2048, 256, rate=rates[0])
        self.aspp2 = ASPP_module(2048, 256, rate=rates[1])
        self.aspp3 = ASPP_module(2048, 256, rate=rates[2])
        self.aspp4 = ASPP_module(2048, 256, rate=rates[3])

        self.relu = nn.ReLU()

        self.global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                             nn.Conv2d(2048, 256, 1, stride=1, bias=False),
                                             nn.BatchNorm2d(256),
                                             nn.ReLU())

        self.conv1 = nn.Conv2d(1280, 256, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(256)

        # adopt [1x1, 48] for channel reduction.
        self.conv2 = nn.Conv2d(256, 48, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(48)

        self.last_conv = nn.Sequential(nn.Conv2d(304, 256, kernel_size=3, stride=1, padding=1, bias=False),
                                       nn.BatchNorm2d(256),
                                       nn.ReLU(),
                                       nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False),
                                       nn.BatchNorm2d(256),
                                       nn.ReLU(),
                                       nn.Conv2d(256, n_classes, kernel_size=1, stride=1),
                                       nn.Sigmoid()
                                      )

    def forward(self, input):
        x, low_level_features = self.resnet_features(input)
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.upsample(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)

        x = torch.cat((x1, x2, x3, x4, x5), dim=1)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = F.upsample(x, size=(int(math.ceil(input.size()[-2]/4)),
                                int(math.ceil(input.size()[-1]/4))), mode='bilinear', align_corners=True)

        low_level_features = self.conv2(low_level_features)
        low_level_features = self.bn2(low_level_features)
        low_level_features = self.relu(low_level_features)


        x = torch.cat((x, low_level_features), dim=1)
        x = self.last_conv(x)
        x = F.upsample(x, size=input.size()[2:], mode='bilinear', align_corners=True)

        return x
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                               mode='min',	# Loss최소화,최대화 결정
                                                               factor=0.1,	# 학습률 감소율
                                                               patience=5,
                                                               verbose=True,)
        monitor_metric = 'val_loss'
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': monitor_metric  # Specify the metric you want to monitor
            }
        }

    def training_step(self, batch, batch_idx):
        x, y = batch['X'], batch['y']
        outputs = self(x).squeeze(dim=1)
        loss = nn.BCELoss()(outputs, y)
        
        predicted_masks = (outputs > 0.5).to(torch.uint8) # 0아니면 1로 바꾸는
        y = y.to(torch.uint8)
        
        iou = self.jaccard(predicted_masks, y)
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_iou', iou, on_step=True, on_epoch=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch['X'], batch['y']
        outputs = self(x).squeeze(dim=1)
        loss = nn.BCELoss()(outputs, y)
        
        predicted_masks = (outputs > 0.5).to(torch.uint8)
        y = y.to(torch.uint8)
  
        iou = self.jaccard(predicted_masks, y)

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_iou', iou, prog_bar=True)

        return loss
    def freeze_bn(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

def get_1x_lr_params(model):
    """
    This generator returns all the parameters of the net except for
    the last classification layer. Note that for each batchnorm layer,
    requires_grad is set to False in deeplab_resnet.py, therefore this function does not return
    any batchnorm parameter
    """
    b = [model.resnet_features]
    for i in range(len(b)):
        for k in b[i].parameters():
            if k.requires_grad:
                yield k


def get_10x_lr_params(model):
    """
    This generator returns all the parameters for the last layer of the net,
    which does the classification of pixel into classes
    """
    b = [model.aspp1, model.aspp2, model.aspp3, model.aspp4, model.conv1, model.conv2, model.last_conv]
    for j in range(len(b)):
        for k in b[j].parameters():
            if k.requires_grad:
                yield k


In [5]:
# WandbLogger를 초기화
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

wandb.login(key='eed81e1c0a41dd8dd67a4ca90cea1be5a06d4eb0')
wandb_logger = WandbLogger(project='AISW', entity='hcim', name='DeepLabV3+')

model = DeepLabv3_plus(nInputChannels=3, n_classes=1, os=16, pretrained=True, _print=True)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=8,
    verbose=True,
    mode='min'
)

trainer = pl.Trainer(devices=[0],max_epochs=100, logger=wandb_logger, callbacks=[early_stopping_callback])
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 3


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

   | Name            | Type               | Params
--------------------------------------------------------
0  | jaccard         | BinaryJaccardIndex | 0     
1  | resnet_features | ResNet             | 42.5 M
2  | aspp1           | ASPP_module        | 524 K 
3  | aspp2           | ASPP_module        | 4.7 M 
4  | aspp3           | ASPP_module        | 4.7 M 
5  | aspp4           | ASPP_module        | 4.7 M 
6  | relu            | ReLU               | 0     
7  | global_avg_pool | Sequential         | 524 K 
8  | conv1           | Conv2d             | 327 K 
9  | bn1             | BatchNorm2d        | 512   
10 | conv2           | Conv2d             | 12.3 K
11 | bn2             | BatchNorm2d        | 96    
12 | last_conv       | Sequential         | 1.3 M 
--------------------------------------------------------
59.3 M    Trainable params
0         Non-trainable params
59.3 M    Total params
237.357   Total estimated model pa

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 00037: reducing learning rate of group 0 to 1.0000e-05.


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 8 records. Best score: 0.049. Signaling Trainer to stop.


In [6]:
"""
class segDataset(Dataset):
    def __init__(self, data_path, transforms=None):
        # cata path 설정 잘하기
        self.pos_imgs = sorted(glob(data_path + 'Positive/Image/*'))
        self.pos_labels = sorted(glob(data_path + 'Positive/Label/*'))
        self.neg_imgs = sorted(glob(data_path + 'Negative/Image/*'))
        self.neg_labels = sorted(glob(data_path + 'Negative/Label /*'))
        # positive와 negative를 합쳐서 불러오는 코드를 작성
        self.imgs = self.pos_imgs + self.neg_imgs
        self.labels = self.pos_labels + self.neg_labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, item):
        img_path = self.imgs[item]
        label_path = self.labels[item]
        img = cv2.imread(img_path)
        label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)
        label = np.expand_dims(label, axis=2)

        concat = np.concatenate([img, label], axis=2)
        concat = torch.from_numpy(concat)
        concat = concat.permute(2,0,1) # (h,w,c -> c,h,w)

        
        if self.transforms:
            imgs = self.transforms(concat)

        X = imgs[:3].to(torch.float32)
        y = imgs[3].to(torch.float32)
            
        return {'X' : X/255, 'y': y/255}

"""

"\nclass segDataset(Dataset):\n    def __init__(self, data_path, transforms=None):\n        # cata path 설정 잘하기\n        self.pos_imgs = sorted(glob(data_path + 'Positive/Image/*'))\n        self.pos_labels = sorted(glob(data_path + 'Positive/Label/*'))\n        self.neg_imgs = sorted(glob(data_path + 'Negative/Image/*'))\n        self.neg_labels = sorted(glob(data_path + 'Negative/Label /*'))\n        # positive와 negative를 합쳐서 불러오는 코드를 작성\n        self.imgs = self.pos_imgs + self.neg_imgs\n        self.labels = self.pos_labels + self.neg_labels\n        self.transforms = transforms\n        \n    def __len__(self):\n        return len(self.labels)\n        \n    def __getitem__(self, item):\n        img_path = self.imgs[item]\n        label_path = self.labels[item]\n        img = cv2.imread(img_path)\n        label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)\n        label = np.expand_dims(label, axis=2)\n\n        concat = np.concatenate([img, label], axis=2)\n        concat = torc

In [7]:
import random
import numpy as np
data_path = './data/Train/'

def set_seed(seed):
    random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_seed(7)

In [8]:
def tensor_to_image(tensor):
    tensor = tensor.squeeze()
    numpy_image = tensor.cpu().numpy()
    numpy_image = (numpy_image * 255).astype(np.uint8)
    
    if numpy_image.shape[0] == 1: # 흑백 이미지라면 채널 차원을 제거.
        numpy_image = np.squeeze(numpy_image, axis=0)
        
    elif numpy_image.shape[0] == 3: # RGB 이미지라면 채널 차원을 맨 뒤로 이동.
        numpy_image = np.transpose(numpy_image, (1, 2, 0))
    return numpy_image

In [9]:
import torch
from PIL import Image

model.eval()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # (h,w,c -> c,h,w) + Normalize
])
def inference_and_save_images(input_dir, output_dir, start_index, end_index):
    for i in range(start_index, end_index + 1):
        input_path = os.path.join(input_dir, f"{i}.jpg")
        output_path = os.path.join(output_dir, f"{i}.jpg")

        # 파일이 존재하지 않으면 스킵. Test/Positive/8322.jpg가 없음
        if not os.path.exists(input_path):
            print(f"File {input_path} not found. Skipping...")
            continue
            
        # 이미지 불러오기 및 전처리
        img = Image.open(input_path)
        img = transform(img).unsqueeze(0)

        # 추론
        with torch.no_grad():
            output = model(img)

        # 텐서를 이미지로 변환후 저장
        output_image = tensor_to_image(output)
        cv2.imwrite(output_path, output_image)

        print(f"Inferred and saved {input_path} to {output_path}")

# Positive 예측
positive_input_dir = "./data/Test/Positive/"
positive_output_dir = "./data/Prediction/Positive/"
inference_and_save_images(positive_input_dir, positive_output_dir, 8000, 8817)

# Negative 예측
negative_input_dir = "./data/Test/Negative/"
negative_output_dir = "./data/Prediction/Negative/"
inference_and_save_images(negative_input_dir, negative_output_dir, 1001, 1181)

Inferred and saved ./data/Test/Positive/8000.jpg to ./data/Prediction/Positive/8000.jpg
Inferred and saved ./data/Test/Positive/8001.jpg to ./data/Prediction/Positive/8001.jpg
Inferred and saved ./data/Test/Positive/8002.jpg to ./data/Prediction/Positive/8002.jpg
Inferred and saved ./data/Test/Positive/8003.jpg to ./data/Prediction/Positive/8003.jpg
Inferred and saved ./data/Test/Positive/8004.jpg to ./data/Prediction/Positive/8004.jpg
Inferred and saved ./data/Test/Positive/8005.jpg to ./data/Prediction/Positive/8005.jpg
Inferred and saved ./data/Test/Positive/8006.jpg to ./data/Prediction/Positive/8006.jpg
Inferred and saved ./data/Test/Positive/8007.jpg to ./data/Prediction/Positive/8007.jpg
Inferred and saved ./data/Test/Positive/8008.jpg to ./data/Prediction/Positive/8008.jpg
Inferred and saved ./data/Test/Positive/8009.jpg to ./data/Prediction/Positive/8009.jpg
Inferred and saved ./data/Test/Positive/8010.jpg to ./data/Prediction/Positive/8010.jpg
Inferred and saved ./data/Test/P

In [10]:
!zip -r Prediction.zip ./data/Prediction

updating: data/Prediction/ (stored 0%)
updating: data/Prediction/Negative/ (stored 0%)
updating: data/Prediction/Negative/1143.jpg (deflated 11%)
updating: data/Prediction/Negative/1094.jpg (deflated 54%)
updating: data/Prediction/Negative/1116.jpg (deflated 14%)
updating: data/Prediction/Negative/1053.jpg (deflated 11%)
updating: data/Prediction/Negative/1087.jpg (deflated 15%)
updating: data/Prediction/Negative/1049.jpg (deflated 15%)
updating: data/Prediction/Negative/1043.jpg (deflated 22%)
updating: data/Prediction/Negative/1065.jpg (deflated 11%)
updating: data/Prediction/Negative/1007.jpg (deflated 43%)
updating: data/Prediction/Negative/1020.jpg (deflated 2%)
updating: data/Prediction/Negative/1077.jpg (deflated 13%)
updating: data/Prediction/Negative/1089.jpg (deflated 8%)
updating: data/Prediction/Negative/1039.jpg (deflated 24%)
updating: data/Prediction/Negative/1015.jpg (deflated 3%)
updating: data/Prediction/Negative/1140.jpg (deflated 41%)
updating: data/Prediction/Negat

In [11]:
import torch

# 모델 정의 및 학습

# 모델 저장
path = "DeepLabV3+.pt"
torch.save(model.state_dict(), path)